steps of a genetic algorithm:

initialization
evaluation
elitism
parent selection
reproduction
mutation

candlestick based genalg params:
modify these based off of largely expanded featurespace

popsize - 200
num gens - 9
elitism copys - 1
mutation prob - 5%
fresh pattern prob - 2%
min number of patterns - 2.5%

In [1]:
import _9_utility as util

dataset = util.load_large_csv('spx_features.csv')


Trying to load CSV file into DataFrame...
loaded chunk 1 of size: 103800164
loaded chunk 2 of size: 103800164
loaded chunk 3 of size: 103800164
loaded chunk 4 of size: 103800164
loaded chunk 5 of size: 103800164
loaded chunk 6 of size: 103800164
loaded chunk 7 of size: 103800164
loaded chunk 8 of size: 103800164
loaded chunk 9 of size: 103800164
loaded chunk 10 of size: 103800164
loaded chunk 11 of size: 103800164
loaded chunk 12 of size: 103800164
loaded chunk 13 of size: 103800164
loaded chunk 14 of size: 92415380
concat chunks
concatted chunks
Success.
Size of dataset:	1441815380


In [2]:
fssr = util.fn_all_subsets(True,0,True)
fssd = util.fnsubset_to_indexdictlist(dataset.columns,fssr)
fss = []
for d in fssd:
    fss.append(list(d.keys()))
print(fss)

[[0, 1, 2, 4, 3, 5, 6, 151, 152, 153], [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66], [67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126], [127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150], [154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212], [213, 214, 215, 216, 217, 218, 219

In [3]:
dataset = dataset.values
arr_low = dataset[:,1]
arr_close = dataset[:,2]

In [4]:
#example for initiation and evaluation steps

from importlib import reload
import _0_gene as _0
reload(_0)
import _1_initialization as _1
reload(_1)
import _2_evaluation as _2
reload(_2)
import _3_elitism as _3
import _4_parentselection as _4
reload(_4)
import _5_reproduction as _5
reload(_5)
import _6_mutation as _6
import _9_utility as util


#dataset = util.load_large_csv('spx_features.csv')
#dataset, fss = util.drop_all_targets()


lag_allowance = 30
generations = 10
hold_for = 10


population = _1.generate_initial_population(
    sample_size=250,
    pattern_size=2,
    feat_idx_pool=fss,
    lag_allowance=lag_allowance,
    skew_lag_prob=False
)

print(len(population[0]._patterns))

print('beginning generational loop')

#begin generational loop
for generation in range(generations):

	print(f'checking population fitness: GEN # {generation+1}')

	returns, kelsch_ratio = _2.fitness(
		arr_close=arr_close,
		arr_low=arr_low,
		data = dataset,
		genes= population,
		hold_for=hold_for,
		lag_allow=lag_allowance,
		specific_data='form_519',
		log_normalize=False
	)

	print('associating population fitness',end='')

	unsorted_population = _2.associate(
		genes=population,
		returns=returns,
		kelsch_ratio=kelsch_ratio
	)

	print('sorting population')

	criteria = 'kelsch_ratio'

	population = _2.sort_population(
		population=unsorted_population,
		criteria=criteria
	)

	avg, top = _2.simple_generational_stat_output(population,criteria)
	print(f"Generation {generation+1} ({criteria}): AVG {avg}, BEST {top}, FROM {len(population)} GENES.")

	print('collecting elites..',end='')

	elites = _3.collect_elite(
		sorted_population=population,
		filter_criteria=1
	)

	print('collecting parents..',end='')


	parents = _4.collect_parents(
		sorted_population=population,
		criteria=criteria,
		num_parents=25
	)

	print('reproducing..',end='')

	family = _5.reproduce(
		parents=parents,
		mode='linear'
	)

	print('shuffling population..',end='')

	shuffled_family = _1.shuffle_population(
		population=family
	)

	print('mutating family',end='')

	#print(f'parents: LEN={len(parents)}')
	#print(f'family: LEN={len(family)}')


	mutated_family = _6.mutation_round(
		shuffled_population=shuffled_family,
		partial_mutation_prob=0.000,
		pattern_mutation_prob=0.000,
		feat_idx_pool=fss	
	)

	print('combining populations')

	population = _1.combine_populations(
		populations=[elites, mutated_family]
	)

2
beginning generational loop
checking population fitness: GEN # 1
associating population fitness

/home/logankelsch/ES/-ES-NN-test1/.tf_/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/logankelsch/ES/-ES-NN-test1/.tf_/lib/python3.12/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


sorting population
Generation 1 (kelsch_ratio): AVG nan, BEST 100.0, FROM 250 GENES.
collecting elites..collecting parents..reproducing..shuffling population..mutating familycombining populations
checking population fitness: GEN # 2
associating population fitnesssorting population
Generation 2 (kelsch_ratio): AVG nan, BEST 100.0, FROM 51 GENES.
collecting elites..collecting parents..reproducing..shuffling population..mutating familycombining populations
checking population fitness: GEN # 3
associating population fitness

KeyboardInterrupt: 

In [ ]:
import _0_gene as _0
import _9_utility as  util
reload(_0)
reload(util)
reload(_2)

_2.show_best_gene_patterns(elites,criteria,fssd)